In [2]:
pip install fasttext


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp311-cp311-linux_x86_64.whl size=4313470 sha256=4b06f0c4dc2e7a448a9f9c6208bafe2edfd4e0652e82dc97fe869549cfda32b3
  Stored in directory: /root/.cache/pip/wheels/65/4f/35/5057db0249224e9ab55a513fa6b79451473ceb7713017823c3
Successfully built fasttext


In [39]:
pip install textaugment nltk


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1


In [3]:
pip install git+https://github.com/facebookresearch/fastText.git


  Cloning https://github.com/facebookresearch/fastText.git to /tmp/pip-req-build-nac_j8_p
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fastText.git /tmp/pip-req-build-nac_j8_p
  Resolved https://github.com/facebookresearch/fastText.git to commit 1142dc4c4ecbc19cc16eee5cdd28472e689267e6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fasttext: filename=fasttext-0.9.2-cp311-cp311-linux_x86_64.whl size=4313396 sha256=5449f96162cc1cf4613931051aff4a7425b859165b222f968e2dfb1cef6704ab
  Stored in directory: /tmp/pip-ephem-wheel-cache-a841t5fw/wheels/04/64/26/11ce8db1ddfa20541eeec84e6969a9d7582367261378c65307
Successfully built fasttext
  Attempting uninstall: fasttext
    Found existing installation: fasttext 0.9.3
    Uninstalling fasttext-0.9.3:
      Successfully uninstalled fasttext-0.9.3


In [41]:
pip install textblob textaugment


In [66]:
import numpy as np
import pandas as pd
import random
import nltk
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Embedding, Conv1D, GlobalMaxPooling1D, Input, concatenate, BatchNormalization, GaussianNoise
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import wordnet

# Download necessary NLTK data
nltk.download('wordnet')

# Load dataset
data = pd.read_csv('filipino_resumes_with_diverse_cover_letters.csv')

# **Balance dataset**
min_class_size = data['Role'].value_counts().min()
data = data.groupby('Role', group_keys=False).apply(lambda x: x.sample(n=min_class_size, random_state=42)).reset_index(drop=True)

# Convert education to binary (1 = Masters, 0 = Others)
data['Education'] = data['Education'].apply(lambda x: 1 if x == 'Masters' else 0)

# Convert experience and age to numeric
data['Experience'] = pd.to_numeric(data['Experience (Years)'], errors='coerce')
data['Age'] = pd.to_numeric(data['Age'], errors='coerce')

# Map roles to numeric labels
role_map = {'Chemical Engineer': 0, 'Accountant': 1}
data['Role'] = data['Role'].map(role_map)

# One-hot encode target variable (Role)
y = to_categorical(data['Role'], num_classes=2)

# Features: Resume data
X_resume = data[['Age', 'Experience', 'Education']]

# Train-test split
X_train_res, X_test_res, y_train, y_test = train_test_split(X_resume, y, test_size=0.3, random_state=42, stratify=y)

# **Standardize resume features** & add noise
scaler = StandardScaler()
X_train_res = scaler.fit_transform(X_train_res) + np.random.normal(0, 0.1, X_train_res.shape)  # ⬅️ Add noise
X_test_res = scaler.transform(X_test_res)

# Save scaler
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# **Text Augmentation**
def synonym_replacement(text, n=2):
    """Replace `n` random words with synonyms using WordNet."""
    words = text.split()
    for _ in range(n):
        word_idx = random.randint(0, len(words) - 1)
        synonyms = wordnet.synsets(words[word_idx])
        if synonyms:
            words[word_idx] = synonyms[0].lemmas()[0].name().replace("_", " ")
    return " ".join(words)

def augment_text(text):
    """Randomly apply synonym replacement (50% of training set)."""
    return synonym_replacement(text, n=2) if random.random() < 0.5 else text

# Apply augmentation only to training data
data.loc[:X_train_res.shape[0], 'Cover Letter'] = data.loc[:X_train_res.shape[0], 'Cover Letter'].apply(augment_text)

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Cover Letter'])

# Save tokenizer
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Convert cover letters to sequences
X_cover = tokenizer.texts_to_sequences(data['Cover Letter'])

# Pad sequences for consistency
X_cover = pad_sequences(X_cover, padding='post', maxlen=50)

# Train-test split for cover letter data
X_train_cover, X_test_cover = train_test_split(X_cover, test_size=0.3, random_state=42, stratify=y)

# **Build the Model (Better Regularization)**
resume_input = Input(shape=(X_train_res.shape[1],), name='resume_input')
x = Dense(8, activation='swish')(resume_input)  # Reduce neurons ⬇️
x = BatchNormalization()(x)
x = Dropout(0.8)(x)  # ⬆️ Increase dropout
x = GaussianNoise(0.1)(x)  # Add Gaussian noise

cover_input = Input(shape=(X_train_cover.shape[1],), name='cover_input')
y = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=32)(cover_input)  # Reduce embedding size ⬇️
y = Conv1D(filters=16, kernel_size=3, activation='swish')(y)  # Reduce filters ⬇️
y = GlobalMaxPooling1D()(y)
y = Dense(8, activation='swish')(y)

combined = concatenate([x, y])

# Output layer (2 roles: Chemical Engineer or Accountant)
z = Dense(2, activation='sigmoid')(combined)  # Change from softmax to sigmoid

# Compile model
model = Model(inputs=[resume_input, cover_input], outputs=z)
model.compile(optimizer=Adam(learning_rate=0.00005),  # Reduce learning rate ⬇️
              loss=BinaryCrossentropy(label_smoothing=0.2),  # Increase label smoothing
              metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train model
model.fit([X_train_res, X_train_cover], y_train,
          epochs=9,  # ⬇️ Reduce epochs to avoid overfitting
          batch_size=16,  # ⬇️ Reduce batch size for better generalization
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping])

# Save model
model.save('model.h5')

# **Evaluate Model**
y_pred = model.predict([X_test_res, X_test_cover])
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Display classification report and accuracy
print("Classification Report:")
print(classification_report(y_test_classes, y_pred_classes))

print("Accuracy Score:", accuracy_score(y_test_classes, y_pred_classes))

# Save label encoders
role_encoder = LabelEncoder()
role_encoder.fit(data['Role'])
with open('label_encoders.pkl', 'wb') as f:
    pickle.dump(role_encoder, f)

print("✅ Model, scalers, and encoders saved successfully!")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
<ipython-input-66-98507714d3b0>:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby('Role', group_keys=False).apply(lambda x: x.sample(n=min_class_size, random_state=42)).reset_index(drop=True)


Epoch 1/9
137/137 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.5164 - loss: 1.0848 - val_accuracy: 0.5082 - val_loss: 0.7439
Epoch 2/9
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5396 - loss: 1.0657 - val_accuracy: 0.5100 - val_loss: 0.7860
Epoch 3/9
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5617 - loss: 1.0281 - val_accuracy: 0.5228 - val_loss: 0.8047
Epoch 4/9
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6013 - loss: 1.0032 - val_accuracy: 0.5811 - val_loss: 0.7810
Epoch 5/9
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6830 - loss: 0.9225 - val_accuracy: 0.6612 - val_loss: 0.7225
Epoch 6/9
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7369 - loss: 0.8625 - val_accuracy: 0.8361 - val_loss: 0.6439
Epoch 7/9
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8338 - loss: 0.7715 - val_accuracy: 0.8634 - val_loss: 0.5689
Epoch 8/9
137/137 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8657 - loss: 0.7219 - val_accuracy: 0.9016 - v

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.88      1.00      0.94       587
           1       1.00      0.87      0.93       588

    accuracy                           0.93      1175
   macro avg       0.94      0.93      0.93      1175
weighted avg       0.94      0.93      0.93      1175

Accuracy Score: 0.934468085106383
✅ Model, scalers, and encoders saved successfully!


In [52]:
import pandas as pd
data = pd.read_csv("filipino_resumes_with_diverse_cover_letters.csv")
print(data['Role'].value_counts())

Role
Accountant           1958
Chemical Engineer    1958
Name: count, dtype: int64


In [3]:
# Save the entire model
model.save("model.h5")


NameError: name 'model' is not defined

In [33]:
import pickle

# Assuming label_encoders is a dictionary of LabelEncoders used for categorical columns
with open("label_encoders.pkl", "wb") as f:
    pickle.dump(label_encoders, f)

In [1]:
pip install python-dotenv


In [2]:
import secrets
print(secrets.token_urlsafe(24))


9J8-Yj5_J989bv25dKiGGYNO1SuMxyQU


In [29]:
import tensorflow as tf
model = tf.keras.models.load_model("model.h5")
print(model.summary())


Model: "functional_77"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ cover_input (InputLayer)  │ (None, 44)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ resume_input (InputLayer) │ (None, 3)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_4 (Embedding)   │ (None, 44, 100)        │         11,700 │ cover_input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_72 (Dense)          │ (None, 32)             │            128 │ resume_input[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_6 (LSTM)             │ (None, 32)             │         17,024 │ embedding_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_38 (Dropout)      │ (None, 32)             │              0 │ dense_72[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_39 (Dropout)      │ (None, 32)             │              0 │ lstm_6[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_73 (Dense)          │ (None, 16)             │            528 │ dropout_38[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_74 (Dense)          │ (None, 16)             │            528 │ dropout_39[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_4             │ (None, 32)             │              0 │ dense_73[0][0],        │
│ (Concatenate)             │                        │                │ dense_74[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_75 (Dense)          │ (None, 3)              │             99 │ concatenate_4[0][0]    │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 30,009 (117.23 KB)

 Trainable params: 30,007 (117.21 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

None


In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
